In [1]:
## ODE
import numpy as np
import pandas as pd # for data manipulation
import time
from scipy.integrate import odeint, solve_ivp
from sklearn.metrics import mean_squared_error
import scipy.optimize as optimize

from ipynb.fs.full.myfun_nn import *
from ipynb.fs.defs.myfun_plot import *
from ipynb.fs.full.myfun_model_nn import *

2023-12-21 16:28:18.000067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 16:28:18.000101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 16:28:18.001104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 16:28:18.006617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 16:28:18.775503: W tensorflow/compiler/tf2

# Useful functions

In [2]:
def time_discretization(t0, tend, deltat=0.05):
    """
    Discretize a time span from t0 to tend with a specified time step.

    Args:
    - t0: Initial time.
    - tend: End time.
    - deltat: Time step (default is 0.05).

    Returns:
    - tspan: Array of discretized time points.
    """

    Nt = round((tend - t0) / deltat) + 1    # Calculate the number of discretization points
    tspan = np.linspace(t0, tend, int(Nt))  # Generate the time span using np.linspace
    
    return tspan

In [3]:
def seq2scn(df):
    """
    Convert a DataFrame of scenes into a list of scenes.

    Args:
    - df: A pandas DataFrame containing scenes.

    Returns:
    - seq: A list of scenes.
    """
    
    seq = []

    # Iterate over rows to extract input and target for each scene
    for row in df.iterrows():
        scn = row[1]     # Extract the scene from the row
        seq.append(scn)  # Append the scene to the list

    return seq

In [4]:
def match_timestamps_scene(t, x, deltat=0.05):
    """
    Match the computed solution timestamps to the same timestamps of the scene data.

    Args:
    - t: Computed solution timestamps.
    - x: Computed solution trajectories.
    - deltat: Time step (default is 0.05).

    Returns:
    - t_matched: Matched timestamps.
    - x_matched: Matched trajectories.
    """
    
    # Calculate the factor for matching based on the time step
    factor = int(0.2 / deltat)

    # Match timestamps and trajectories using slicing
    t_matched = np.array(t)[::factor]
    x_matched = np.array([traj[::factor] for traj in x])

    return t_matched, x_matched

In [5]:
def update_sol_lists(N, tspan_ann, sol_ann, x_list, t_list):
    """
    Update lists containing time and trajectory data after solving an ODE in a sub-interval of a scene.

    Args:
    - N: Number of vehicles.
    - tspan_ann: Timestamps for the sub-interval solution.
    - sol_ann: Computed solution for the sub-interval.
    - x_list: List of vehicle trajectories.
    - t_list: List of timestamps.

    Returns:
    - x_list: Updated list of vehicle trajectories.
    - t_list: Updated list of timestamps.
    """

    # Convert the solution and timestamps to lists
    x_ann = sol_ann.tolist()
    t_ann = tspan_ann[1:]  # Exclude the first recording

    # Add the solution to the correct vehicle trajectory
    for j in range(0, N):
        tmp = x_ann[j][1:]  # Exclude the first recording
        x_list[j] = np.concatenate([x_list[j], tmp])
    
    t_list = np.concatenate([t_list, t_ann]).tolist()

    return x_list, t_list

In [ ]:
def nn_odesolver(model_nn, N, xspan, tspan, normalize_flag=False):
    """

    """

    # SOLVE ODE with ANN
    t_list, sol_ann_list, v_list = [tspan[0]], [xspan], []

    for i in range(len(tspan)-1):

        # Guess for the ODE
        x0 = sol_ann_list[i]        
        tstart, tend = tspan[i], tspan[i+1] 

        cd_ann = np.abs(np.diff(x0, axis=0))    
        y_pred = model_nn(cd_ann, training=False)

    #     print(fr"At it={i}, v={y_pred}")
    #     print("\n")

        v0 = np.max(y_pred)*1.1 #artificio
        v_ann = y_pred.numpy().flatten().tolist() + [v0]
        
        # normalizing...
        if normalize_flag:
            v_ann = v_ann/np.max(v_ann)

    #     print(fr"At it={i}, v={v_ann}")
    #     print("\n")

        tspan_ann = np.linspace(tstart, tend, 5)  # Generate the time span using np.linspace 
        sol_ann = odeint(TD_ANN_model, x0, tspan_ann, args=(v_ann,), tfirst=True).T

        # Store the information
        t_list.append(tspan_ann[-1])
        lastpos = [sol_ann[iveh][-1] for iveh in range(N)]
        sol_ann_list.append(lastpos)
        v_list.append(v_ann)

    # to reach the same len for all the three lists
    cd_ann = np.abs(np.diff(sol_ann_list[-1], axis=0))    
    y_pred = model_nn(cd_ann, training=False)
    v0 = np.max(y_pred)*1.1 #artificio
    v_ann = y_pred.numpy().flatten().tolist() + [v0]
    
    # normalizing...
    if normalize_flag:
        v_ann = v_ann/np.max(v_ann)
    
    v_list.append(v_ann)

    ## Post
    # transposing...
    sol_ann_list = list(map(list, zip(*sol_ann_list)))
    v_list = list(map(list, zip(*v_list)))
    # making them as a numpy obj
    sol_ann_list = np.array(sol_ann_list)
    v_list = np.array(v_list)

    return t_list, sol_ann_list, v_list